In [560]:
from datetime import date,timedelta
import datetime
import math
import pyodbc
import os, sys
import re
import pandas as pd
import numpy as np
import pyodbc
from sqlalchemy import create_engine
server = 'bieno-da-d-80166-unilevercom-sql-01.database.windows.net'
database = 'bieno-da-d-80166-unilevercom-sqldb-01'
username = 'PROD_LoadPool'
password = 'Dlk45^8&&dodsslk12Ad7*as'
driver = 'SQL Server Native Client 11.0'
port = '1433'
pd.options.mode.chained_assignment = None
data_directory = 'C:/Users/tristan.dresbach/OneDrive - Unilever/'

In [561]:
#get fullfilled orders:
#want to get all orders where quantity expected = quantity dispatched (aka the order was fullfilled in full) for a given date range
#the date range is manual as the busisness likes this to be run on an ad hoc basis.
sql ='''
select MaterialNumber,SalesOrderCode,CustomerSoldTo,ReportingDate, QuantityExpected,QuantityDispatched from RawReporting_BDL.FactSalesOrderLine
where ReportingDate >= '2021-03-01' and ReportingDate < '2021-03-24' and QuantityOrdered = QuantityDispatched and QuantityOrdered <> 0 and SalesOrganisationCode in ('0001')
'''
#EDIT BACK SALES ORGS: '0002','0003','0007'
connection_string = 'DRIVER={};SERVER={};PORT={};DATABASE={};UID={};PWD={}'.format(
    driver, server, port, database, username, password)
cnxn = pyodbc.connect(connection_string)
good_orders = pd.read_sql(sql, cnxn)  # Can be assigned to any variable name
cnxn.close()

In [525]:
#in case you want to run the code for a subset of items, you can import a list of DUs and filter the "good_orders" df and the "orders_cut" df
#that is run below for your selected input items

#tgt_items = pd.read_excel('Book1.xlsx')
#tgt_items['DU'] = [(18-len(str(x)))*'0' + str(x) for x in tgt_items['DU']]
#good_orders = good_orders[good_orders['MaterialNumber'].isin(tgt_items['DU'].tolist())]
#good_orders = good_orders.reset_index(drop=True)

In [563]:
#create our cuts column to be used later on

good_orders['LBD_QT'] = good_orders['QuantityExpected'] - good_orders['QuantityDispatched']

In [564]:
#get cut orders:
#the cut column is "(LI.QuantityLossBeforeDispatch*-1) [LBD_QT]", we use the same date range as for the previous "good_orders" SQL query.
# last line of SQL query: "AND LI.QuantityLossBeforeDispatch <> 0" guarantees that we are only pulling cuts

sql ='''
    SELECT
            LI.MaterialNumber
            , LI.[SalesOrderCode]
            , LI.CustomerSoldTo
            , LI.ReportingDate
            , LI.QuantityExpected
            ,LI.QuantityDispatched

           
           ,(LI.QuantityLossBeforeDispatch*-1) [LBD_QT]
           
        FROM [RawReporting_BDL].[FactSalesOrderLine] LI
        INNER JOIN [TeradataConnect].[DimCalendar] CAL ON LI.ReportingDate = CAL.Date_Dt
        INNER JOIN [RawReporting_BDL].[DimSalesOrganisation] AS SO ON LI.SalesOrganisationCode =SO.SalesOrganisationCode
        INNER JOIN [RawReporting_BDL].[DimDistributionChannel] AS DC ON LI.DistributionChannelCode =DC.DistributionChannelCode
        INNER JOIN [RawReporting_BDL].[HierarchyCordilleraLocalProduct] AS PS ON LI.MaterialNumber =PS.MaterialNumber AND LI.SalesOrganisationCode=PS.SalesOrganisationCode AND LI.DistributionChannelCode=PS.DistributionChannel
        WHERE CAL.Days_From_Current > 0 and CAL.Date_Dt >= '2021-03-01' and CAL.Date_dt < '2021-03-24
        AND LI.DistributionChannelCode IN ('10','61') ---different distribution channels, 
        AND LI.SalesOrganisationCode IN ('0001')
        AND LI.UnitOfMeasureCode = 'CS'
        AND PS.LocalLevel3Code NOT IN ( '006', '0FF', '008', '04D', '0F1')
        AND LI.QuantityLossBeforeDispatch <> 0
'''
#'0002','0003','0007'
connection_string = 'DRIVER={};SERVER={};PORT={};DATABASE={};UID={};PWD={}'.format(
    driver, server, port, database, username, password)
cnxn = pyodbc.connect(connection_string)
orders_cut = pd.read_sql(sql, cnxn)  # Can be assigned to any variable name
cnxn.close()

In [510]:
#these two lines below are part of the same process to filter the data for the items in which we are interested from that input file

#orders_cut = orders_cut[orders_cut['MaterialNumber'].isin(tgt_items['DU'].tolist())]
#orders_cut = orders_cut.reset_index(drop=True)

In [565]:
orders = pd.concat([good_orders,orders_cut],ignore_index=True)
orders.rename(columns={'LBD_QT':'Cuts','ReportingDate':'date'},inplace=True)

In [566]:
#list of sold tos, for names:
customers = list(set(orders['CustomerSoldTo']))
customers_str = str(customers)

In [567]:
#get all customer names, using the list of soldtos we just generated from our data:
sql ='''
select CustomerCode,EccCustomerLevel5Name from RawReporting_BDL.HierarchyCordilleraCustomer
where CustomerCode in ''' + customers_str.replace('[','(').replace(']',')')
connection_string = 'DRIVER={};SERVER={};PORT={};DATABASE={};UID={};PWD={}'.format(
    driver, server, port, database, username, password)
cnxn = pyodbc.connect(connection_string)
cus_names = pd.read_sql(sql, cnxn)  # Can be assigned to any variable name
cnxn.close()

In [569]:
#append customer names and merge our base df ("orders") with the customer related data:
cus_names.rename(columns={'CustomerCode':'CustomerSoldTo','EccCustomerLevel5Name':'Customer'},inplace=True)
cus_names = cus_names.drop_duplicates(subset=['CustomerSoldTo'],keep='first').reset_index(drop=True)
df = pd.merge(orders,cus_names,how='left',on='CustomerSoldTo')

In [570]:
#prep df for analysis:
#we filter out weird data issues like DUs that have empty spaces, weird amount of characters, are = -1
#create key column for future looping as well

df_f = df.copy()
#df_f = df_f.sort_values(['MaterialNumber','date']).reset_index(drop=True)
df_f = df_f[~df_f['MaterialNumber'].str.contains(' ')].reset_index(drop=True)
df_f = df_f[~df_f['MaterialNumber'].str.len() < 5].reset_index(drop=True)
#df_f = df_f[df_f['date'] <= date.today()]
#df_f = df_f[(df_f['QuantityDispatched'] >= 0) & (df_f['QuantityOriginal'] > 0)].reset_index(drop=True)
df_f = df_f[df_f['MaterialNumber'] != '-1']
df_f['key'] = df_f['MaterialNumber'] + '_' + df_f['CustomerSoldTo']

In [597]:
#in case we want to run our analysis for a specific customer, we can use this filter and input the customer name

#df_f = df_f[df_f['Customer'] == 'TARGET']

In [572]:
#setup output:
#want to view the data at a DU & lane level hence we do a groupby to know how much should have shipped and wasn't
#we create a list of keys from this that will be the DU/lane we are interested in looping through to identify true demand 
output_df = df_f.groupby(['MaterialNumber','CustomerSoldTo','Customer','key'],as_index=False)[['QuantityExpected','Cuts']].sum()
output_df.drop_duplicates(inplace=True)
output_df = output_df[output_df['QuantityExpected'] > 1]
output_df = output_df.sort_values(['Cuts'],ascending=False)
output_df = output_df.reset_index(drop=True)
df_f_m = df_f[df_f['key'].isin(list(output_df['key']))]

In [575]:
#setup data only for cuts:
#make sure that the df we loop through contains the previous list of created unique keys, and then groupby date so that POs
#that occured on the same day are see as one smooth PO, helps for the looping
study_keys = df_f_m['key'].unique().tolist()
study_df = df_f_m[df_f_m['key'].isin(study_keys)]
study_df = study_df[['date','QuantityExpected','QuantityDispatched','Cuts','key']]
study_df = study_df.groupby(by=['date','key']).sum().reset_index()

In [576]:
#makes sure for the key in question, we have more than one day of orders. If we only have one day of orders, we can't tell if it is
#true demand or not. Create a list of good keys "keys_goo_len" to loop through our logic:
keys_good_len = []
for key in study_keys:
    key_df = study_df[study_df['key'] == key]
    if len(key_df) > 1:
        keys_good_len += [key]

In [613]:
#initialize variables that we will use to track the output throughout our looping:
all_keys = []
all_order_qs = []
all_dates = []
true_demand = []

#create function to find if value is in bounds of order quantity and order frequency, identifies which is the issue (could be both):
def bounds(q,high_q,low_vol_q,order_indx):
    viola = []
    if q > high_q:
        viola += ['order']
    if diff_days[order_indx-1] < low_vol_q:
        viola+= ['freq']
    return viola


#for key in study_keys[0:99]:
#for key in keys_good_len[0:99]:
for key in ['000000000067903092_0030003959']:
    
#loop through each key in our good key list:
#for key in keys_good_len:
    print(key)
    #filter df to the key we are looping, get all dates and the key listed the amount of times as the key_df is long to track output
    key_df = study_df[study_df['key'] == key]
    key_df = key_df.sort_values(by=['date'])
    all_dates += key_df['date'].tolist()
    all_keys += key_df['key'].tolist()
    
    #calcualte the difference between days, for order frequency study later
    days = sorted(set(key_df['date']))
    init_day = min(key_df['date'])
    days_list = [(x - init_day).days for x in days]
    
    #initialize main variables, cuts, orders and dispatched quantity:
    cuts_list = []
    order_list = []
    disp_list = []
    for i in range(len(key_df)):
        cuts_list += [key_df['Cuts'].tolist()[i]]
        order_list += [key_df['QuantityExpected'].tolist()[i]]
        disp_list += [key_df['QuantityDispatched'].tolist()[i]]
        
    #create list of orders that are 100% true:
    #we use our true orders as the metrics for what we expect our order quantity and order frequency to be
    true_orders = []
    n = 0
    cut_ct = 0
    post_days_counter = 20
    true_orders_idx = []
    
    #calculate what the average, low and high order frequencies are. We keep in cut orders as we want a pessemistic estimate and
    #we need enough data points to make this work
    diff_days = [y - x for x,y in zip(days_list[:-1],days_list[1:])]
    avg_order_days = np.mean(diff_days)
    std_order_days = round(np.array(diff_days).std(),2)
    std_mult_d = 1.0
    high_vol = round(avg_order_days + std_mult_d*std_order_days,1)
    low_vol = round(avg_order_days - std_mult_d*std_order_days,1)
    if low_vol < 0:
        low_vol = 1
    
    #how far away from a cut order does a fullfilled order have to be to consider that fullfilled order as NOT correlated to the cut:
    bound_true_orders  = round(2*avg_order_days,0)
    
    #identify true orders:
    #to identify true orders, we take a look at the below 4 scenarios. We study if it was a cut, and how many days after a cut
    #did the order in question we are looking at happen.
    while n < len(cuts_list):
        #if no cuts and no leading up cuts:       
        if ((cuts_list[n] == 0) & (post_days_counter >= bound_true_orders)):
            true_orders += [order_list[n]]
            true_orders_idx += [n]
        #if no leading up cuts and we have cuts:
        if ((cuts_list[n] > 0) & (post_days_counter >= bound_true_orders)):
            true_orders += [order_list[n]]
            post_days_counter = 0
            true_orders_idx += [n]
        #if we have cuts and there are leading up cuts:    
        if ((cuts_list[n]>0) & (post_days_counter < bound_true_orders)):
            post_days_counter = 0
        #if we have no cuts but leading up cuts:
        if ((cuts_list[n] == 0) & (post_days_counter < bound_true_orders)):
            if n+1 != len(cuts_list):
                day_diff = days_list[n+1] - days_list[n]
                post_days_counter += day_diff      
        n += 1
    
    #calculate order metrics:
    #need to calculate our low and high bounds for what we expect as a true order:
    avg_order_q = np.mean(true_orders)
    std_order_q = round(np.array(true_orders).std(),2)
    std_mult_q = 2.0
    high_order_q = round(avg_order_q + std_mult_q*std_order_q,0)
    low_order_q = round(avg_order_q - std_mult_q*std_order_q,0)
    if low_order_q < 0:
        low_order_q = 1

    
    true_orders_mod = []
    mod_indx = []
    for indx in range(len(cuts_list)):
        #make sure we are not checking true orders:
        if indx not in true_orders_idx:
            mod_indx += [indx]
            order = order_list[indx]
            #we check to see for this not true order, what kind of violations it has (either frequency 'freq' or order quantity related)
            violations = bounds(order,high_order_q,low_vol,(indx))
            if len(violations)>0:
                print(indx,violations)
                print(diff_days[indx-1],low_vol,order,high_order_q)
                if len(violations) ==1:
                    if violations[0] == 'freq':
                        #if we have a order frequency violation, we adjust the order quantity down, by the ration between the 
                        #frequency of this not true demannd order and the low bound of what is acceptable for order frequency
                        true_orders_mod += [round(diff_days[indx-1]/low_vol*order,0)]
                    else:
                        #if not a frequency issue, adjust the order q to the max of our acceptable bounds
                        true_orders_mod += [high_order_q]
                else:
                    #if we are here, then both the freq and order q are an issue, so we adjust for both here:
                    true_orders_mod += [round(diff_days[indx-1]/low_vol*high_order_q,0)]
            else:
                true_orders_mod += [order]
    #re-arrange modified order quantities and add them to our tracking lists
    for n in range(len(key_df)):
            if n in true_orders_idx:
                all_order_qs += [true_orders[true_orders_idx.index(n)]]
                true_demand += [1]
            else:
                all_order_qs += [true_orders_mod[mod_indx.index(n)]]
                true_demand += [0]
        
    
    


000000000067903092_0030003959


ValueError: min() arg is an empty sequence

In [579]:
#create df with output lists from logic:
temp = pd.DataFrame()
temp['key'] = all_keys
temp['Adjusted true demand'] = all_order_qs
temp['date'] = all_dates
temp['Is QE true demand?'] = true_demand
#merge our newly create df with the true demand columns with the columns from the intially generated dfs
temp_full = study_df[study_df['key'].isin(temp['key'].tolist())]
final_df = pd.merge(temp_full,temp,how='left',on=['key','date'])
final_df = final_df.sort_values(by=['key','date']).reset_index(drop=True)

In [581]:
all_adj_cuts = []
#need to loop through each unique key in our final df to compensate for change in order quantities
#as we have modified order quantirties, the cut quantities and quantity dispatched need to be modified accordingly
for key in final_df['key'].unique().tolist():
    #create base variables that will be the basis of our logic:
    subset_df = final_df[final_df['key'] == key]
    QE = subset_df['QuantityExpected'].tolist()
    Cuts = subset_df['Cuts'].tolist()
    adjD = subset_df['Adjusted true demand'].tolist()
    QD = subset_df['QuantityDispatched'].tolist()
    running_sum = 0
    #loop through each day of orders:
    for i in range(len(subset_df)):
        #if we have no cuts and quantity expected is the same as adjusted, no modifications
        if (Cuts[i] ==0) and (QE[i] == adjD[i]):
            all_adj_cuts += [0]
        #if we have no cuts and quantity expefcted is different as adjusted quantity, cuts are still 0 but we have "extra"
        #hypotehtical quantities, so we "store" that quantity to decrease later cuts
        if (Cuts[i] == 0) and (QE[i] != adjD[i]):
            running_sum += QE[i] - adjD[i]
            all_adj_cuts += [0]
            
        if (Cuts[i] != 0):
            #if we have cuts, we check to see if we have previously "stored" quantities, if not, no problem. If we do,
            #we use those stored quantities to decrease cuts (in the first else loop)
            if running_sum == 0:
                cut_diff = adjD[i] - QD[i]
                cut_q_add = Cuts[i] - cut_diff
                running_sum += cut_q_add
                all_adj_cuts += [cut_diff]
            else:
                cut_diff = adjD[i] - QD[i]
                if running_sum < cut_diff:
                    cut_diff -= running_sum
                    running_sum = 0
                    all_adj_cuts += [cut_diff]
                else:
                    cut_diff = 0
                    running_sum -= cut_diff
                    all_adj_cuts += [cut_diff]

In [582]:
#make cosmetic changes to our final_df, columns requested by the business
final_df['Adjusted true cuts'] = all_adj_cuts
final_df['Order Q that is over true demand'] = [y-x for x,y in zip(final_df['Adjusted true demand'],final_df['QuantityExpected'])]
final_df['Cut Q that is over true cuts'] = [y-x for x,y in zip(final_df['Adjusted true cuts'],final_df['Cuts'])]
study = final_df.groupby(['key'],as_index = False).sum()
study['DU'] = [x.split('_')[0] for x in study['key']]
study = study.groupby(['DU'],as_index=False).sum()
study = study.sort_values(by=['Order Q that is over true demand'],ascending=False).reset_index(drop=True)

In [592]:
#add in description, UPC and GTIN:
sql ='''

select MaterialNumber as DU, EANOrUPC,MaterialDescription,MaterialGtin from [RawReporting_BDL].DimMaterial '''
#EDIT BACK SALES ORGS: '0002','0003','0007'
connection_string = 'DRIVER={};SERVER={};PORT={};DATABASE={};UID={};PWD={}'.format(
    driver, server, port, database, username, password)
cnxn = pyodbc.connect(connection_string)
join_data = pd.read_sql(sql, cnxn)  # Can be assigned to any variable name
cnxn.close()

full_study = pd.merge(study,join_data,how='left',on='DU').reset_index()
full_study['percentage of total'] = [  round((100*x/sum(full_study['Order Q that is over true demand'])),2)  for x in full_study['Order Q that is over true demand']]

In [596]:
#export to excel for business use:
#full_study.to_excel('study_tgt_all_items_c.xlsx')